<a href="https://colab.research.google.com/github/patricksabry1/42028-Deep-Learning/blob/master/42028_Assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Pre-Processing

## Load the dataset and store into train/test split variables

In [0]:
import tensorflow as tf
# Load train and test datasets 
def loadData():
    mnist = tf.keras.datasets.mnist
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train, x_test = x_train / 255.0, x_test / 255.0

    print('X_train shape:', X_train.shape)
    print('Number of images in x_train:', X_train.shape[0])
    print('Number of images in x_test:', X_test.shape[0])

    return (x_train, y_train), (x_test, y_test)

X_train shape: (60000, 28, 28, 1)
Number of images in x_train: 60000
Number of images in x_test: 10000


## Histogram-of-Oriented Gradient feature

## Local Binary Patterns feature

In [0]:
from skimage import feature
import numpy as np
import cv2

# Get LBP features for the dataset
class LocalBinaryPatterns:
	def __init__(self, numPoints, radius):
		# store the number of points and radius
		self.numPoints = numPoints
		self.radius = radius
 
	def LBPfeatures(self, image, eps=1e-7):
		# compute the Local Binary Pattern representation
		# of the image, and then use the LBP representation
		# to build the histogram of patterns
		lbp = feature.local_binary_pattern(image, self.numPoints,
			self.radius, method="uniform")
        # Form the histogram
		(hist, _) = np.histogram(lbp.ravel(),
			bins=np.arange(0, self.numPoints + 3),
			range=(0, self.numPoints + 2))
 
		# normalize the histogram
		hist = hist.astype("float")
		hist /= (hist.sum() + eps)
 
		# return the histogram of Local Binary Patterns
		return hist

def getLBPFeatures():
    # Create an object of LocalBinaryPatterns class and initial the parameters.
    (x_train, y_train), (x_test, y_test) = loadData()
    desc = LocalBinaryPatterns(24, 8)
    lbp_train = []
    lbp_labels_train = []

    # loop over the training images
    for img_index in range(len(lbp_train)):
        # load the train image, and extract LBP features
        image = cv2.imread(x_train[img_index])
        # convert image to grayscale
        grayscale_img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 
        hist = desc.describe(grayscale_img)

        # extract the label from the image path, then update the
        # label and data lists
        lbp_labels_train.append(y_train[img_index])
        lbp_train.append(hist)
    
    return lbp_labels_train, lbp_train

# SVM

In [0]:
from sklearn.svm import LinearSVC

lbp_labels_train, lbp_train = getLBPFeatures()

model = LinearSVC(C=100.0, random_state=42)
model.fit(lbp_train, lbp_labels_train)

# CNN

## Define & compile neural network + fit on dataset

In [0]:
# Define CNN model with
    # Declare input shape for first layer of CNN
    input_shape = (28, 28, 1)
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=input_shape),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])
model.summary()

model.fit(x_train, y_train, epochs=20)


ERROR! Session/line number was not unique in database. History logging moved to new session 59
Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_4 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 128)               100480    
_________________________________________________________________
dropout_4 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 10)                1290      
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2931 - accuracy: 0.9151
Epoch 2/20
1875/1875 [===========================

## Evaluate trained model on validation data

In [0]:
model.evaluate(x_test,  y_test, verbose=2)


313/313 - 1s - loss: 0.0858 - accuracy: 0.9800


[0.08582599461078644, 0.9800000190734863]

# KNN